<a href="https://colab.research.google.com/github/Ivan8Garcia/Proyecto_NoCountry/blob/main/Vuelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

datos=pd.read_csv("/content/flight_delays.csv")
datos.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/flight_delays.csv'

In [ ]:
datos.info()

In [ ]:
datos.isnull().sum()

In [ ]:
datos.fillna(0,inplace=True)

In [ ]:
datos.isnull().sum()

In [ ]:
datos.head()

##ELIMINACION DE COLUMNAS PARA MODELO MVP, SOLO NECESITAREMOS "Aerolínea","Origen", "Destino", "Hora_salida_programada", "Hora_llegada_programada","Retraso","Distancia" y crear la columna binaria "Retrasado".

In [ ]:
import pandas as pd
import numpy as np

# Re-read data to ensure this cell works with the original columns
# This effectively resets 'datos' to the state before other preprocessing cells.
datos = pd.read_csv("/content/flight_delays.csv")
datos.fillna(0, inplace=True) # Apply previous fillna logic

# Select and rename columns as per original intent of this cell, plus ScheduledArrival for user's request
datos = datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture','ScheduledArrival','Distance','DelayMinutes']].copy()
datos = datos.rename(columns={'Airline': 'Aerolínea',
                              'Origin': 'Origen',
                              'Destination': 'Destino',
                              'ScheduledDeparture': 'Fecha/Hora_Salida',
                              'ScheduledArrival': 'Fecha/Hora_Llegada_Programada',
                              'Distance' : 'Distancia',
                              'DelayMinutes': 'Retraso_minutos'})

# Process 'Fecha/Hora_Salida' (original 'ScheduledDeparture')
datos['Fecha/Hora_Salida'] = pd.to_datetime(datos['Fecha/Hora_Salida'])

# Add day of the week
datos['Dia_semana_temp'] = datos['Fecha/Hora_Salida'].dt.day_name()
spanish_days = {
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado', 'Sunday': 'Domingo'
}
datos['Dia_semana'] = datos['Dia_semana_temp'].map(spanish_days)
datos.drop('Dia_semana_temp', axis=1, inplace=True)

# Extract 'Fecha' and 'Hora_salida'
datos['Fecha'] = datos['Fecha/Hora_Salida'].dt.date
datos['Hora_salida'] = datos['Fecha/Hora_Salida'].dt.time
datos.drop('Fecha/Hora_Salida', axis=1, inplace=True) # Drop the original combined column

# Process 'Fecha/Hora_Llegada_Programada' (original 'ScheduledArrival') as requested
# Convert to datetime first
datos['Fecha/Hora_Llegada_Programada'] = pd.to_datetime(datos['Fecha/Hora_Llegada_Programada'])

# Extract the time part into a new column, e.g., 'Hora_llegada'
datos['Hora_llegada'] = datos['Fecha/Hora_Llegada_Programada'].dt.time

# "elimines la fecha" from 'Fecha/Hora_Llegada_Programada'.
# This means we either drop the column or make it store only time.
# If we keep the column and make it store only time, it becomes redundant with 'Hora_llegada'.
# So, it's best to drop 'Fecha/Hora_Llegada_Programada' now that its time part is extracted.
datos.drop('Fecha/Hora_Llegada_Programada', axis=1, inplace=True)

# Create the binary 'retrasado' column based on 'Retraso_minutos'
datos['Retrasado'] = (datos['Retraso_minutos'] > 15).astype(int)

# Ensure the requested column order, placing 'Retraso_minutos' and 'Retrasado' after 'Hora'
datos = datos[['Aerolínea', 'Origen', 'Destino','Distancia', 'Dia_semana', 'Fecha', 'Hora_salida','Hora_llegada', 'Retraso_minutos', 'Retrasado']]

datos.head()

In [ ]:
datos.drop("Fecha",axis=1,inplace=True)
datos.drop("Dia_semana",axis=1,inplace=True)

In [ ]:
datos.head()

In [ ]:
datos.info()

In [ ]:
# Convert 'Hora' from datetime.time object to a float representing total hours
datos['Hora_salida'] = datos['Hora_salida'].apply(lambda x: x.hour + x.minute/60 + x.second/3600)
datos['Hora_llegada'] = datos['Hora_llegada'].apply(lambda x: x.hour + x.minute/60 + x.second/3600)

datos.info()

In [ ]:
datos.head(3)

In [ ]:
datos.tail(3)

In [ ]:
#variables categoricas
datos.describe(include="O")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
avg_delay= datos.groupby("Aerolínea")["Retraso_minutos"].mean().reset_index()
sns.barplot(x="Aerolínea",y="Retraso_minutos",data=avg_delay)
plt.title("Compañias Aereas vs Retraso promedio")
plt.xlabel("Compañias Aereas")
plt.ylabel("Retraso promedio en minutos")
plt.show()

sns.countplot(data=datos,x="Aerolínea")
plt.title("Numero de vuelos por compañia aerea")
plt.xlabel("Compañia Aerea")
plt.ylabel("Numero de vuelos")
plt.show()

In [ ]:
order=datos["Aerolínea"].value_counts().index
sns.countplot(data=datos,x="Aerolínea",order=order)
plt.title("Numero de vuelos por Aerolínea")
plt.xticks(rotation=70)
plt.xlabel("Compañia")
plt.ylabel("Numero de vuelos")
plt.show()

In [ ]:
delay_counts = datos['Retrasado'].value_counts(normalize=True)

plt.figure(figsize=(8, 8))
plt.pie(delay_counts, labels=['No Retrasado (0)', 'Retrasado (1)'], autopct='%1.1f%%', startangle=90, colors=['#66b3ff', '#ff9999'])
plt.title('Proporción de Vuelos Retrasados vs. No Retrasados')
plt.ylabel('') # Hide the default 'Retrasado' label on the y-axis
plt.show()

##**Metodo get_dummies**

In [ ]:
#Todas estas columnas son categoricas (OBJ), para que se puedan hacer buenos modelos hay que codificarlas
#categoricas= ["Aerolínea","Origen","Destino"]

#pd.get_dummies(data=datos,columns=categoricas,dtype=int).head(3)

In [ ]:
#datos_codificados=pd.get_dummies(data=datos,columns=categoricas,dtype=int)
#datos_codificados.sample(3)

In [ ]:
#datos_codificados.info()

In [ ]:
#df=datos_codificados
#df

##**OneHotEnconder**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


##**✅ Definir X e y, observese que se excluye Retraso_minutos**

In [ ]:
# Variable objetivo
y = datos["Retrasado"]

# Variables predictoras
X = datos[[
    "Aerolínea",
    "Origen",
    "Destino",
    "Distancia",
    "Hora_salida",
    "Hora_llegada"
]]


##**✅  Crear variables cíclicas para las horas**

In [ ]:
def cyclical_encode_hour(datos, col_name):
    datos[col_name + "_sin"] = np.sin(2 * np.pi * datos[col_name] / 24)
    datos[col_name + "_cos"] = np.cos(2 * np.pi * datos[col_name] / 24)
    return datos

datos = cyclical_encode_hour(datos, "Hora_salida")
datos = cyclical_encode_hour(datos, "Hora_llegada")

# Eliminamos columnas originales de hora
datos = datos.drop(columns=["Hora_salida", "Hora_llegada"])


In [ ]:
datos.head()

##**✅  Separar columnas categóricas y numéricas**

In [ ]:
categorical_features = ["Aerolínea", "Origen", "Destino"]

numeric_features = [
    "Distancia",
    "Hora_salida_sin",
    "Hora_salida_cos",
    "Hora_llegada_sin",
    "Hora_llegada_cos"
]


##**✅  Transformador de columnas (One-Hot incluido)**

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)


##**✅  Definir el modelo (puedes cambiarlo si quieres)**

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)


##**✅  Crear Pipeline completo**

In [64]:
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", model)
])


##**✅ Train/Test Split**

In [65]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


##**✅ Entrenamiento**

In [66]:
clf.fit(X_train, y_train)


ValueError: A given column is not a column of the dataframe

##**✅ Evaluacion**

In [ ]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


In [ ]:
X = datos.drop('Retrasado', axis = 1)
y = datos['Retrasado']


In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

categoricas= ["Aerolínea","Origen","Destino"]

one_hot_enc = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),
    categoricas),
    remainder='passthrough',
    sparse_threshold=0,
    force_int_remainder_cols=False)

datos = one_hot_enc.fit_transform(datos)
datos = pd.DataFrame(datos, columns=one_hot_enc.get_feature_names_out())
datos

In [ ]:
columnas = X.columns
X = one_hot_enc.fit_transform(X)
X = pd.DataFrame(X, columns=one_hot_enc.get_feature_names_out(columnas))

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,  random_state = 42)

##**Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(max_depth = 10, random_state=5)
rf.fit(pd.DataFrame(X_train), y_train)
rf.score(X_test,y_test)

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Define X and y specifically for the classification task
# X should exclude both 'Retraso_minutos' (which is the regression target)
# and 'Retrasado' (which is the classification target).
X = df.drop(["Retrasado","Retraso_minutos"], axis=1)
y = df["Retraso_minutos"]

# Split the data for the classification task
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Instanciar el DummyClassifier
# Usamos 'most_frequent' porque es el baseline más común: siempre predice la clase mayoritaria.
dummy_clf = DummyClassifier(strategy="most_frequent")

# 2. "Entrenar" el modelo (aquí solo aprende cuál es la clase más frecuente)
dummy_clf.fit(X_train, y_train)

# 3. Realizar predicciones sobre el set de prueba
y_pred = dummy_clf.predict(X_test)

# 4. Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del Baseline: {accuracy:.2f}")

# 5. Ver el reporte detallado
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['No Retrasado', 'Retrasado'], zero_division=0))

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

In [ ]:
#primero separamos nuestras columnas para tener las columnas de prueba y entrenamiento
X= df.drop(["Retrasado"],axis=1)
y=df["Retrasado"]

#segundo,empezaremos a segmentar nuestras X_train,etc...
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

baseline=DummyRegressor()
baseline.fit(X_train,y_train)

In [ ]:
#ahora vamos a evaluar el modelo usando metricas
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

y_pred_dummy=baseline.predict(X_test)

def calcular_regresion (y_test,y_pred):
  rmse=mean_squared_error(y_test,y_pred)
  mae=mean_absolute_error(y_test,y_pred)
  r2= r2_score(y_test,y_pred)
  metricas={
      "RMSE":round(rmse**(1/2),4),
      "MAE":round(mae,4),
      "R2":round(r2,4)
  }
  return metricas

In [ ]:
resultados_baseline=calcular_regresion(y_test,y_pred_dummy)
resultados_baseline

## Interpretación de Métricas de Regresión

*   **Interpretación:** Mide la magnitud promedio de los errores del modelo. La unidad del RMSE es la misma que la de la variable objetivo (en tu caso, los minutos de retraso). Valores más bajos indican un mejor ajuste del modelo a los datos.
*   **¿Qué es un 'buen' valor?** No hay un umbral fijo para un 'buen' RMSE, ya que depende en gran medida del contexto y la escala de tus datos. Sin embargo, un RMSE significativamente menor que la desviación estándar de la variable objetivo suele ser una buena señal. Es una métrica sensible a errores grandes (outliers) debido al cuadrado de los errores.
*   **¿Es una buena métrica?** Sí, es ampliamente utilizada. penaliza más los errores grandes, lo que puede ser deseable si los errores grandes son particularmente problemáticos en tu aplicación. Su principal desventaja es que, al ser sensible a outliers, un solo error muy grande puede inflar el RMSE.

### 2. Error Absoluto Medio (MAE - Mean Absolute Error)

*   **¿Qué es?** El MAE es el promedio de la magnitud de los errores. A diferencia del RMSE, no eleva al cuadrado los errores, sino que toma su valor absoluto.
*   **Interpretación:** También mide la magnitud promedio de los errores del modelo. Su unidad es la misma que la de la variable objetivo. Valores más bajos indican un mejor rendimiento.
*   **¿Qué es un 'buen' valor?** Similar al RMSE, depende de la escala de tus datos. Un MAE menor que el rango de tu variable objetivo y, si es posible, menor que un umbral de error aceptable en tu dominio, es deseable.
*   **¿Es una buena métrica?** Sí, es otra métrica muy común. Es más robusta a los outliers que el RMSE porque no penaliza los errores grandes de forma cuadrática. Ofrece una interpretación más directa de la magnitud promedio del error en las unidades originales de la variable objetivo. Si prefieres que todos los errores contribuyan por igual a la métrica, MAE es una buena elección.

### 3. Coeficiente de Determinación (R2 - R-squared)

*   **¿Qué es?** El R2 es una medida de la proporción de la varianza en la variable dependiente que es predecible a partir de las variables independientes. En términos más simples, indica qué tan bien se ajustan las predicciones de tu modelo a los valores reales.
*   **Interpretación:** El valor de R2 varía entre 0 y 1 (o puede ser negativo para modelos muy malos, como en tu caso para el baseline).:
    *   **R2 = 0:** El modelo no explica ninguna de la variabilidad de la variable objetivo. Las predicciones del modelo no son mejores que simplemente usar el promedio de los valores reales.
    *   **R2 = 1:** El modelo explica el 100% de la variabilidad de la variable objetivo. El modelo predice perfectamente los valores reales.
    *   **R2 negativo:** Esto sucede cuando el modelo que has ajustado es peor que un modelo que simplemente predice la media de la variable dependiente para todas las observaciones. En tu caso, `R2: -0.0` para el `DummyRegressor` es esperado, ya que este modelo trivial solo predice la media, y la forma en que se calcula R2 lo hace ligeramente negativo si la suma de cuadrados de los residuos es mayor que la suma de cuadrados totales.
*   **¿Qué es un 'buen' valor?** En general, cuanto más cercano a 1, mejor. Sin embargo, un R2 alto no siempre significa que el modelo sea práctico, especialmente si se ha sobreajustado a los datos de entrenamiento. Un 'buen' R2 también es contextual y depende de la complejidad del problema. En ciencias sociales, un R2 de 0.3 puede ser considerado bueno, mientras que en física, se esperan valores mucho más altos.
*   **¿Es una buena métrica?** Sí, es muy útil para entender la bondad de ajuste global del modelo. La precaución principal es no usarlo como la única métrica, ya que un R2 alto no garantiza un modelo preciso o sin sesgos, y puede ser engañoso en presencia de muchos predictores.

### Resumen para tus resultados `{'RMSE': 11.8311, 'MAE': 10.2462, 'R2': -0.0}`:

Estos resultados son de tu modelo *baseline* (`DummyRegressor`). Un modelo *baseline* es un punto de partida simple y sirve para comparar. Al ser un `DummyRegressor`, es normal que obtengas un R2 muy bajo o negativo, ya que es un modelo muy básico que probablemente predice solo la media de los minutos de retraso. El MAE y RMSE de tu baseline te indican el error promedio de este modelo trivial.

Para saber si un futuro modelo es bueno, querrías ver:

*   **RMSE y MAE:** Significativamente más bajos que los obtenidos con el modelo *baseline*.
*   **R2:** Un valor positivo y, idealmente, cuanto más cercano a 1, mejor (aunque con cautela y comparando con otros modelos).

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calcular_metricas_regresion(y_test, y_pred):

    rmse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    metricas = {
        'Raíz del Error Cuadrático Medio': round(rmse, 4),
        'Error Absoluto Medio': round(mae, 4),
        'R2 Score': round(r2, 4)
    }

    return metricas

In [ ]:
from sklearn.dummy import DummyRegressor

estrategias = ["mean", "median", ("quantile", 0.25), ("constant", 10.0)]

resultados = {}

for estrategia in estrategias:
    if isinstance(estrategia, tuple):
        estrategia, valor = estrategia
        if estrategia == "quantile":
            model_dummy = DummyRegressor(strategy=estrategia, quantile=valor)
        else:
            model_dummy = DummyRegressor(strategy=estrategia, constant=valor)
    else:
        model_dummy = DummyRegressor(strategy=estrategia)

    model_dummy.fit(X_train, y_train)

    y_pred_dummy = model_dummy.predict(X_test)

    metricas = calcular_metricas_regresion(y_test, y_pred_dummy)

    resultados[estrategia] = metricas

for estrategia, metricas in resultados.items():
    print(f"Estrategia: {estrategia}")
    for metrica, valor in metricas.items():
        print(f"{metrica}: {valor}")
    print("\n")


##Metodo Random Forest

In [ ]:
#los resultados son en minutos; aqui vemos que el modelo generalizó mejor
#RMSE= 6.51min, MAE=5.48min, 'R2': 0.6966%
from sklearn.ensemble import RandomForestRegressor

modelo= RandomForestRegressor(max_depth=5,random_state=42)
modelo.fit(X_train,y_train)
y_pred=modelo.predict(X_test)
resultados_rf= calcular_regresion(y_test,y_pred)
resultados_rf

## **Cross Validation**

In [ ]:
from sklearn.model_selection import KFold,cross_validate

scoring={
    "RMSE":"neg_root_mean_squared_error",
    "MAE": "neg_mean_absolute_error",
    "R2": "r2"
}
cv= KFold(n_splits=5,shuffle=True,random_state=42)
cv_resultados= cross_validate(modelo,X_train,y_train,scoring=scoring,cv=cv)
cv_resultados


In [ ]:
len(df.columns)

In [ ]:
modelo.feature_importances_

In [ ]:
importances= modelo.feature_importances_

In [ ]:
feature_importances=pd.DataFrame({"Features":X.columns,"Importances":(importances*100).round(2)}).sort_values("Importances",ascending=False)
feature_importances

In [ ]:
resultados=pd.DataFrame(index=["RMSE","MAE","R2"])
model_features=RandomForestRegressor(max_depth=5,random_state=42)
ct_features= [i if i !=0 else 1 for i in range(0,17,4)]

for i in ct_features:
  selected_features= feature_importances["Features"].values[:i]
  X_train_sel= X_train[selected_features]
  X_test_sel= X_test[selected_features]
  model_features.fit(X_train_sel,y_train)
  y_pred=model_features.predict(X_test_sel)
  metricas=calcular_regresion(y_test,y_pred)
  resultados[i]= list(metricas.values())

resultados

In [ ]:
resultados=pd.DataFrame(index=["RMSE","MAE","R2"])
model_features=RandomForestRegressor(max_depth=5,random_state=42)
ct_features=  range(1,5)

for i in ct_features:
  selected_features= feature_importances["Features"].values[:i]
  X_train_sel= X_train[selected_features]
  X_test_sel= X_test[selected_features]
  model_features.fit(X_train_sel,y_train)
  y_pred=model_features.predict(X_test_sel)
  metricas=calcular_regresion(y_test,y_pred)
  resultados[i]= list(metricas.values())

resultados

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rf= RandomForestClassifier(max_depth = 10, random_state=42)
#rf.fit(pd.DataFrame(X_train), y_train)
#rf.score(X_test,y_test)

##**Metodo OneHotEnconder**